In [1]:
# -*- coding:utf-8 -*-
%matplotlib inline 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
tm_vin=pd.read_csv('tm_vin.csv')
vin_vehicleCode=pd.read_csv('vin.csv')
version=pd.read_csv('feature_list_version.csv')
feature=pd.read_csv('tm_feature.csv')

IOError: File tm_vin.csv does not exist

In [ ]:
#this is our data from shanghai insurence company
#few of them may be wrong,but just a few.
print len(vin_vehicleCode)
vin_vehicleCode.head(10)

In [ ]:
#this is the EPC from the company 
print len(tm_vin)
tm_vin.head(10)

In [ ]:
#this is what we want to show to the insurane company
vehicle_code=pd.read_csv('vehicle_code.csv')
vehicle_code.head(10)

In [ ]:
print vehicle_code['VEHICLE_CODE'].is_unique
print 'we have different kinds of vehicle code from the EPC:',len(vehicle_code)

In [ ]:
#the number of vehicle code from the original data of insurance company
vin4=vin_vehicleCode.groupby('VEHICLE_CODE',as_index=False)['VEHICLE_MODEL'].count()

print 'how many vehicle codes  from the insurance company:',len(vin4)
#the number of vehicle code which have no more than 10 cars for each vehicle code
# print 'how many vehicle code which have no more than 10 cars from the insurance company:',len(vin4[vin4['VEHICLE_MODEL']<=1])
# print vin4.sort('VEHICLE_MODEL')

# easy code to show 
# print vin_vehicleCode['VEHICLE_CODE'].value_counts()
print 'how many vehicle code which have no more than 10 cars from the insurance company:',np.sum(vin_vehicleCode['VEHICLE_CODE'].value_counts()<2)
print 'this data will cause problme when trainning'

In [ ]:
vin_vehicleCode

In [ ]:
#check wheather all the vehicle code from the insurance company is in the EPC
# all(xx in vin4['VEHICLE_CODE'] if xx in vehicle_code)

In [ ]:
# let us have a look at the feature list version csv and the feature list
feature_list_version= pd.read_csv('FEATURE_LIST_VERSION.csv')
print 'we have different features:',len(feature_list_version)
feature_list_version.head(20)

In [ ]:
print feature_list_version['FEATURE_LIST'][0]
print 'the length of feature list',len(feature_list_version['FEATURE_LIST'][0])

In [ ]:
#now we want to interpret the feature list to mapping the vehicle code

#1-4:   2     year
#5-8:   3     export country
#4bits: 5     colour 
#4bits: 6     inside colour
#4bits: 7     vin
#3-4bits:A00  car style

tm_feature = pd.read_csv('tm_feature.csv')
tm_feature[0:50]

In [ ]:
# we want to have a look at the number of levels for each family code(categorical feature)
b = tm_feature.groupby('FAMILY_CODE',as_index=False).count()
b

In [ ]:
print type(b)
print 'we have different features from the feature list corresponding to each vin code: ',len(b)

In [ ]:
## filter the feature with family code

#b['FAMILY_CODE']
# i=0
# # b['FAMILY_CODE'][i]
# i=i+1
# print b['FAMILY_CODE'][i]
# tm_feature[tm_feature['FAMILY_CODE']==b['FAMILY_CODE'][i]]
#tm_feature[tm_feature['FAMILY_CODE']=='A10']

In [ ]:
##check the information from the insurance company and the EPC database
#WE want to use this information to construct a tranning data: featuers(from feature list vesrsion id to feature list,then to 
#feature) and Vehicle_Code(as lable)
vin_conbined=pd.merge(vin_vehicleCode,tm_vin,left_on='RACK_NO',right_on='VIN_CODE',how='inner')
print 'the num of obsevations from the insurance compay acturall in our EPC database:',len(vin_conbined), '\n'
print 'we have the obsevations from the insurance company:',len(vin_vehicleCode)
#some of the rack_code from the insurance company is not exist in our EPC database
vin_conbined.head(10)

In [ ]:
#we want to try to find the vehicle code with the vin code from the insurance company
#if it is in the EPC database:
#but no vehicle code is directly matched, we want to predict the vehicle code by classification method with the corresponding 
#featues.so we can search in the EPC database to find the feature list by vin code and interpret it as the correspongding features
#which can help us to predict the vehicle code we had.

#first,we need to construct a trainning data from the data of insurance company,and trainning a 'good' algorithm to predict:
#interpret the feature list into lots of features for each car from insurance company, then make it as trainning data to construct some alogrithm to predict 


#but if the vin code from the insurance company is not in the EPC database(becaues of the delay of database):
#we cannt get the featuers from our EPC database,then we have to predict the vehicle code just by vin code. one way is just treat the vin code as one singel feature to predict,
#another way is to interpret the vin code into meanningful features to predict.(the question is how?)

In [ ]:
# vin2=vin1[['VEHICLE_CODE','FEATURE_LIST_VERSION_ID']]
# print len(vin2)
# vin2.head(10)

In [ ]:
# vin2['num']=0
# vin2.head(10)

In [ ]:
# vin3=vin1.groupby(['FEATURE_LIST_VERSION_ID','VEHICLE_CODE'],as_index=False)['VEHICLE_MODEL'].count()
# print len(vin3)
# vin3.head(10)
#we find the vehicle code is not unique for each feature list version 

#vin3['num']=vin3['num'].astype(int)
#vin3[['FEATURE_LIST_VERSION_ID','VEHICLE_CODE','NUM']].head(10)

In [ ]:
#now we want to construct a trainning data with 79389 obsevations (actually in the EPC database)and 349 features and the vehicle code as label
vin_conbined.head()
raw_data =vin_conbined[['RACK_NO','VEHICLE_CODE','VEHICLE_MODEL','PLATFORM_CODE','FEATURE_LIST_VERSION_ID']]
print len(raw_data)
raw_data =pd.merge(raw_data,feature_list_version,on='FEATURE_LIST_VERSION_ID')
print  'construct a trainning data with 79389 obsevations (actually in the EPC database',len(raw_data)
raw_data.head()

In [ ]:
#set the column of the train data
dict_column = tm_feature.groupby('FAMILY_CODE',as_index=False).count()['FAMILY_CODE']
dict_column= list(dict_column)
print type(dict_column)
print len(dict_column)
#print dict_column


#set the empty train data frame
train_data = pd.DataFrame(columns=dict_column)
train_data_CN = pd.DataFrame(columns=dict_column)
print train_data

In [ ]:
# coding:utf-8
#then we want to interpret the feature list into 349 features
print  'we show one sample to interpre the feature list to 349 features:'
sample = raw_data['FEATURE_LIST'][0]
print sample
type(sample)
len(sample)
sample[4:8]
length =len(sample)

temp = tm_feature[tm_feature['FEATURE_CODE']==sample[0:0+4]]['FEATURE_CN_NAME']
print temp[temp.index[0]]
# print type(tm_feature[tm_feature['FEATURE_CODE']==sample[0:0+4]]['FEATURE_CN_NAME'])
# print tm_feature[tm_feature['FEATURE_CODE']==sample[0:0+4]]['FEATURE_CN_NAME'] # it output a series with index
#     index =tm_feature[tm_feature['FEATURE_CODE']==sample[0:0+4]].index

for x in range(0,len(sample),4):
         print x, sample[x:x+4],
         print tm_feature[tm_feature['FEATURE_CODE']==sample[x:x+4]]['FEATURE_CN_NAME']      

In [ ]:
#samle is a feature list
temp_data = pd.DataFrame(np.nan, index=[0],columns=dict_column)
temp_data_CN = pd.DataFrame(np.nan, index=[0],columns=dict_column)

feature_list = raw_data['FEATURE_LIST'][0]
# print type(feature_list)
# print feature_list
# print temp_data
# print type(dict_column)
# print type(feature_list[0])
count =0
for xx in range(0,len(feature_list),4):
    
    if (feature_list[xx] in ['2','3','5','6','7']) and (count <=6):
        count = count+1
        feature =feature_list[xx]
        feature_code =feature_list[xx:xx+4]
        #print xx,count,feature_code
        #print feature_code
        
        if tm_feature[tm_feature['FEATURE_CODE']==feature_code].values.shape[0]==0:
            #some feature_code like 76E3 in 279th observation ,but not exsit in the tm_feature['FEATURE_CODE], we assume it is just the correce 
            #feature_code,but fail to find the chinese feature name
            if tm_feature[tm_feature['FEATURE_CODE']==feature_code[0:3]].values.shape[0]==0:
                temp_data[feature] = np.nan
                temp_data_CN[feature]= np.nan
                          
            #some feature_code lke G10N in 0th observation, but not exsit in the tm_feature['FEATURE_CODE],
            #we assume it is the same as G10    
            else:
                temp_data[feature] = feature_code[0:3]
                temp_data_CN[feature]= feature_code[0:3]
         
        #print feature_code
        else:
            temp_data[feature] = feature_code
            #print feature_code, tm_feature[tm_feature['FEATURE_CODE']==feature_code]
            temp_data_CN[feature]= tm_feature[tm_feature['FEATURE_CODE']==feature_code].values[(0,4)]
            #.values which change a data frame into np array

    elif any(feature_list[xx:xx+3] == x for x in dict_column):
        #print any(feature_list[xx:xx+3] == x for x in dict_column)
        feature = feature_list[xx:xx+3]
        feature_code = feature_list[xx:xx+4]
        
        if tm_feature[tm_feature['FEATURE_CODE']==feature_code].values.shape[0]==0:
            #some feature_code like 76E3 in 279th observation ,but not exsit in the tm_feature['FEATURE_CODE], 
            #2017/7/18 now I find the error is come from the transformation from database to csv, 
            #76E3 will be transformed into scientific notation 7.60E+04
            #76E9 will be transformed into scientific notation 7.60E+10
            
            # 76E6    7.60E+07
            #74E3 will be transformed into scientific notation 7.40E+04
            # 74E4    7.40E+05


            #so when we transform the data fromthe database to csv file, we need to make sure the data tape as string 
            
            
            
            #we assume it is just the correce 
            #feature_code,but fail to find the chinese feature name
            if tm_feature[tm_feature['FEATURE_CODE']==feature_code[0:3]].values.shape[0]==0:
                print 'feature code is not exist: '+ featrue_code
                temp_data[feature] = np.nan
                temp_data_CN[feature]= np.nan
                
                
            #some feature_code lke G10N in 0th observation, but not exsit in the tm_feature['FEATURE_CODE],
            #we assume it is the same as G10    
            else:
                temp_data[feature] = feature_code[0:3]
                temp_data_CN[feature]= feature_code[0:3]
         
        #print feature_code
        else:
            temp_data[feature] = feature_code
            #print feature_code, tm_feature[tm_feature['FEATURE_CODE']==feature_code]
            temp_data_CN[feature]= tm_feature[tm_feature['FEATURE_CODE']==feature_code].values[(0,4)]
            #.values which change a data frame into np array
            
        #print type(tm_feature[tm_feature['FEATURE_CODE']==feature_code]),feature_code
        #print tm_feature[tm_feature['FEATURE_CODE']==feature_code].values.shape[0]
    
                  
    
#     print temp
#     print type(temp)
    #print str(temp[temp.index[0]])# a wierd situation here
    #print temp[0:1]
    #temp_data_CN[feature]= tempCN
    
#the feature code version of feature for one car
print temp_data
#the chinese verion of feature for one car
print temp_data_CN


# we can use other 'FEATURE_CN_CHINA' OR 'NUM' HERE         

# To improve the efficiency of interpret the vin code into the features 


In [ ]:
feature_list_raw_data = raw_data['FEATURE_LIST']

In [ ]:
# #now we want to do the same process to every car from the insurance company to build a train data, extract every car' feature list into a 
# #dataframe 
# num_vehicle = len(raw_data)
# #set the timer
# import timeit
# start_time= timeit.default_timer()

# #test for a few observations
# # num_vehicle_test=2000

# #training for the whole data
# num_vehicle_test = num_vehicle



# for x in range(num_vehicle_test):
#     #set a empty temp_data for each vehicle
#     temp_data = pd.DataFrame(np.nan, index=[0],columns=dict_column)
#     temp_data_CN = pd.DataFrame(np.nan, index=[0],columns=dict_column)
#     #choose the feature list for each vehicle
    
# #     feature_list = raw_data['FEATURE_LIST'][x]
#     feature_list = feature_list_raw_data[x]  # which make you do not need to read the whole table each time
    
#     #extract the features from the feature_list for each vehicle into the dataframe named temp_data or temp_data_CN
#     count =0
    
    
#     for xx in range(0,len(feature_list),4):

#         if (feature_list[xx] in ['2','3','5','6','7']) and (count <=6):
#             count = count+1
#             feature =feature_list[xx]
#             feature_code =feature_list[xx:xx+4]
#             #print xx,count,feature_code
#             #print feature_code

#             if tm_feature[tm_feature['FEATURE_CODE']==feature_code].values.shape[0]==0:
#                 #some feature_code like 76E3 in 279th observation ,but not exsit in the tm_feature['FEATURE_CODE], we assume it is just the correce 
#                 #feature_code,but fail to find the chinese feature name
#                 if tm_feature[tm_feature['FEATURE_CODE']==feature_code[0:3]].values.shape[0]==0:
#                     temp_data[feature] = np.nan
#                     temp_data_CN[feature]= np.nan

#                 #some feature_code lke G10N in 0th observation, but not exsit in the tm_feature['FEATURE_CODE],
#                 #we assume it is the same as G10    
#                 else:
#                     temp_data[feature] = feature_code[0:3]
#                     temp_data_CN[feature]= feature_code[0:3]

#             #print feature_code
#             else:
#                 temp_data[feature] = feature_code
#                 #print feature_code, tm_feature[tm_feature['FEATURE_CODE']==feature_code]
#                 temp_data_CN[feature]= tm_feature[tm_feature['FEATURE_CODE']==feature_code].values[(0,4)]
#                 #.values which change a data frame into np array

#         elif any(feature_list[xx:xx+3] == x for x in dict_column):
#             #print any(feature_list[xx:xx+3] == x for x in dict_column)
#             feature = feature_list[xx:xx+3]
#             feature_code = feature_list[xx:xx+4]

#             if tm_feature[tm_feature['FEATURE_CODE']==feature_code].values.shape[0]==0:
#                 #some feature_code like 76E3 in 279th observation ,but not exsit in the tm_feature['FEATURE_CODE], 
#                 #2017/7/18 now I find the error is come from the transformation from database to csv
                
#                 #we assume it is just the correce feature_code,but fail to find the chinese feature name
#                 if tm_feature[tm_feature['FEATURE_CODE']==feature_code[0:3]].values.shape[0]==0:
#                     temp_data[feature] = np.nan
#                     temp_data_CN[feature]= np.nan


#                 #some feature_code lke G10N in 0th observation, but not exsit in the tm_feature['FEATURE_CODE],
#                 #we assume it is the same as G10    
#                 else:
#                     temp_data[feature] = feature_code[0:3]
#                     temp_data_CN[feature]= feature_code[0:3]

#             #print feature_code
#             else:
#                 temp_data[feature] = feature_code
#                 #print feature_code, tm_feature[tm_feature['FEATURE_CODE']==feature_code]
#                 temp_data_CN[feature]= tm_feature[tm_feature['FEATURE_CODE']==feature_code].values[(0,4)]
#                 #.values which change a data frame into np array
   
#     #test temp_data every 100 obsevations    
#     if x%100==0:
#         print x,temp_data_CN    
#     #append the temp data into the big train data
#     train_data = train_data.append(temp_data)
#     train_data_CN=train_data_CN.append(temp_data_CN)
    
# end_time= timeit.default_timer()
# run_time =end_time-start_time
# print run_time

    

In [ ]:
# run_time = 144652.977632 =1.6742242781509526 days
# #because the data is huge,we want to save it into csv file to resues later
# train_data.to_csv('train_data.csv',encoding='utf-8',index= False)
# train_data_CN.to_csv('train_data_CN.csv',encoding='utf-8',index= False)


# we can read the train data from the file we stored befor then you dont need to run the code above to generate the train data
train_data= pd.read_csv('train_data.csv') 
# when we store a table ,we will store the index into the 1st column,so when we read,we need to tell the computer which column is index column 
# train_data= pd.read_csv('train_data.csv',index_col=0)

train_data_CN =pd.read_csv('train_data_CN.csv')

# saving datafames to an excel workbook
# writer = pd.ExcelWriter('train_data.xlsx')
# train_data.to_excel(writer,'sheet1')
# writer.save()
# train_data.columns
# train_data.drop(train_data.columns[0],inplace= True,axis =1)
train_data.head()

In [ ]:
train_data_CN.head()

In [ ]:
print len(raw_data)
#print num_vehicle_test
print train_data.shape
print train_data_CN.shape
#train_data
#train_data[100:200]

#### check the table we have

In [ ]:
vin_table= pd.read_csv('vin.csv')
vin_table[vin_table['RACK_NO']=='LSJW26H38AS072686']

In [ ]:
tm_vin[tm_vin['VIN_CODE']=='LSJW26H38AS072686']

In [ ]:
feature_list_version[feature_list_version['FEATURE_LIST_VERSION_ID']==1000036275]

In [ ]:
raw_data

In [ ]:
train_data.head()

## Explore data analysis (EDA)

### 1) EDA about the raw data from the insurance compay adn out EPC database

In [ ]:
# We want to inspect the distribution for each plat form
platform_groups = raw_data.groupby('PLATFORM_CODE_x',as_index=True).VEHICLE_CODE.count()
print 'we have {0} kinds of platforms from the insurance company which is in our EPC '.format(len(platform_groups))
platform_groups.plot.bar()

### 2) EDA about the train data with features transformed from the raw data

In [ ]:
print train_data.shape




In [ ]:
train_data.head()
# figure = train_data 

# Part1. Preprocessing (80% of work)
### Agenda:(from April Chen)
1.1. Basic Data Cleaning
    1. Dealing with data types
    2. Handling missing data
1.2. More Data Exploration
    1. Outlier detection
    2. Plotting distributions
1.3. Feature Engineering
    1. Interactions between features
    2. Dimensionality reduction using PCA
1.4. Feature Selection and Model Building




### 1.1. Basic Data Cleaning
### A. Dealing with data types

- There are three main data types:
    - Numeric, e.g. income, age
    - Categorical, e.g. gender, nationality 
    - Ordinal, e.g. low/medium/high
    
    
- Models can only handle numeric features


- Must convert categorical and ordinal features into numeric features (Encoding categorical features)
    - Create dummy features
    - Transform a categorical feature into a set of dummy features, each representing a unique category
    - In the set of dummy features, 1 indicates that the observation belongs to that category

#### 1) use the get_dummies funtion to tranform the categorical variables into dummy variable with binary values

In [ ]:
#show the dummy varibals for the feature '2'(year)
pd.get_dummies(train_data['2'],prefix='2').head(5)

In [ ]:
# Create a list of features to dummy
# todummy_list = list(train_data.columns)
# # Function to dummy all the categorical variables used for modeling
# def dummy_df(df, todummy_list):
#     for x in todummy_list:
#         dummies = pd.get_dummies(df[x], prefix=x, dummy_na=False)
#         df = df.drop(x, 1)
#         df = pd.concat([df, dummies], axis=1)
#     return df

#better than just pd.get_dummies
#X_train_dummies= pd.get_dummies(X_train)

In [ ]:
# X_train_dummies= dummy_df(train_data,todummy_list)

In [ ]:
# print train_data.shape
# print X_train_dummies.shape
# X_train_dummies.head()

#### 2) LabelEncoder and OneHotEncoder
It is used to transform non-numerical labels to numerical labels (or nominal categorical variables). 
Numerical labels are always between 0 and n_classes-1. 
OneHotEncoder takes as input categorical values encoded as integers

In [ ]:
# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# X_train_label =pd.DataFrame()

# for x in X_train.columns:
    
#     X_train_label[x] = le.fit_transform(train_data[x].astype('str'))

# X_train_label.shape
# X_train_label.head() 


In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# enc = OneHotEncoder()
# enc.fit(df_label)  
# enc.transform(df_label).toarray()

### B1) Handling missing data (for dense train data)

- Models can not handle missing data


- Simplest solution
    - Remove observations/features that have missing data
    

- But, removing missing data can introduce a lot of issues
    - Data is randomly missing: potentially lose a lot of your data
    - Data is non-randomly missing: in addition to losing data, you are also introducing potential biases
    - Usually, this is a poor solution


- An alternative solution is to use imputation
    - Replace missing value with another value
    - Strategies: mean, median, highest frequency value of given feature

### B2) Handling missing data (for sparse train data)
how to deal the missing data, simply using imputation here seems problematic!!!

or then treat the train data as sparse and select the features
seletion the important feature by feature selection algorithm??
#### 1) Transformed into Sparse matrix
#### 2) Feature selection
seletion the important feature by feature selection algorithm



In [ ]:
'''
class sklearn.decomposition.SparseCoder(dictionary, transform_algorithm='omp', transform_n_nonzero_coefs=None, transform_alpha=None, split_sign=False, n_jobs=1)[source]

'''

'''
Feature selection is usually used as a pre-processing step before doing the actual learning. 
The recommended way to do this in scikit-learn is to use a sklearn.pipeline.Pipeline:

In this snippet we make use of a sklearn.svm.LinearSVC coupled with sklearn.feature_selection.SelectFromModel to evaluate feature importances 
and select the most relevant features. 
Then, a sklearn.ensemble.RandomForestClassifier is trained on the transformed output, i.e. using only relevant features. 
You can perform similar operations with the other feature selection methods and also classifiers that provide a way to 
evaluate feature importances of course. 
See the sklearn.pipeline.Pipeline examples for more details.
'''

"""
clf = Pipeline([
  ('feature_selection', SelectFromModel(LinearSVC(penalty="l1"))),
  ('classification', RandomForestClassifier())
])
clf.fit(X, y)
"""

## Impoove Features by adding the feature from the vin code

In [ ]:
'''
VIN码详解：
1）1～3位： （WMI：世界制造厂识别代码） 全球制造厂识别码，表示制造厂、品牌和类型；用来标识车辆制造厂的唯一性。通常占VIN代码的前三位；
2) 4～9位（VDS：车辆说明部分）：
说明车辆的一般特性，制造厂不用其中的一位或几位字符，就在该位置填入选定的字母或数字占位，其代号顺序由制造厂确定。
第 4～8位(VDS)：车辆特征
第四位：车身及底盘系列代码
第五位：发动机类型代码
第六～七位：车型代码
第八位：乘员安全保护装置代码

第9位：校验位  通过一定的算法防止输入错误；

3)
第10位：车型年份（ 车型年款代码 ），即厂家规定的型年（Model Year），不一定是实际生产的年份，但一般与实际生产的年份之差不超过1年；
第11位：装配厂（总装工厂代码）；
12～17位： 出厂顺序号代码 ，一般情况下，汽车召回都是针对某一顺序号范围内的车辆，即某一批次的车辆。

'''
# the features we want to extract fromt the vin code
features_from_vin = ['vin_code_no4','vin_code_no5','vin_code_no6','vin_code_no7','vin_code_no8','vin_code_no10','vin_code_no11']

feature_adding = pd.DataFrame(columns=features_from_vin)
#extract the vin code into a series
rack_no = raw_data['RACK_NO']

# # for x in range(len(raw_data)):
for x in range(10):
    print rack_no[x]
    temp=pd.DataFrame(columns=features_from_vin)
   
    for xx in range(len(features_from_vin)):
        temp[temp.columns[xx]]=(rack_no[x][xx+3])
    print temp
    feature_adding.append(temp)
#     pd.concat([feature_adding,temp],axis=0)



In [ ]:
# # combine the fatures from the feature list and vin code
train_data_combined = pd.concat(feature_adding,train_data,axis = 1)
train_data_CN_combined  = pd.concat(feature_adding,train_data_CN,axis=1)

## One Easy way to deal with missing value in the train data and fealture selection
### from the professional opinions


###  Question: what we can do to the categorical variable and missing value?

### (1) check the nan in the train data in each column 

In [ ]:
print train_data.isnull().sum()
train_nan= pd.DataFrame(train_data.isnull().sum()) # tranform the series into a dataframe

#set the threshold for the nan values ,we think the more import the feature is ,the less nan rate it will have 
nan_threshold = len(train_data)*0.1

#show the columuns whose number of nan value is less than 2000, to choose the features in feture
print len(train_nan[train_nan[train_nan.columns[0]]<nan_threshold])
print train_nan[train_nan[train_nan.columns[0]]<nan_threshold]

### (2) seletion the important and non nan features by nan value rate

In [ ]:
train_nan[train_nan[train_nan.columns[0]]< nan_threshold]
feature_list =list(train_nan[train_nan[train_nan.columns[0]]<nan_threshold].index)
feature_list

X_train_nanRate = train_data[feature_list]
#sample train of 2000 observation 
#y_train = raw_data.ix[0:1999,'VEHICLE_CODE']

#the whole train
y_train = raw_data['VEHICLE_CODE']
print X_train_nanRate.shape
print len(y_train)

In [ ]:
'''
C:\ProgramData\Anaconda3\envs\python27\lib\site-packages\sklearn\cross_validation.py:516: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=3.
  % (min_labels, self.n_folds)), Warning)
'''
# some target in y_train just have one observation, which make the algorithm much hard to train
# so we need to find out how many of these observations
#the number of vehicle code from the original data of insurance company



# print  raw_data['VEHICLE_CODE'].value_counts()
print 'how many vehicle codes  from the insurance company ',len(vin_vehicleCode['VEHICLE_CODE'].value_counts())
print 'how many vehicle codes  from the insurance company which also in our EPC databse:',len(raw_data['VEHICLE_CODE'].value_counts())
print 'how many vehicle code which have no more than 3 cars from the insurance company:',np.sum(raw_data['VEHICLE_CODE'].value_counts()<4)



#create the dictonary which have no more than 3 vehicles cresoponding to the vehicle code
dict_countFew =[]
temp_index = raw_data['VEHICLE_CODE'].value_counts().index
temp_count = list(raw_data['VEHICLE_CODE'].value_counts())
for index,count in zip(temp_index,temp_count):
    
    if count < 4:
#         print index
        dict_countFew.append(index)
    else:
        pass
    
len(dict_countFew)

In [ ]:
# find the index of the obsevation which have few vehicle code cresponding
index_countFew =[]
for index,vehicleCode in enumerate(raw_data['VEHICLE_CODE']):
    if vehicleCode in dict_countFew:
        index_countFew.append(index)
    else:
        pass
print 'the number of vehicle whose vehicle code have no more 3 in the train data',len(index_countFew)
print 'we need to remove them from our train data to improve the model'
# index_countFew       

In [ ]:
print len(X_train_nanRate),len(y_train)
print len(X_train_nanRate.drop(index_countFew))
X_train_nanRate=X_train_nanRate.drop(index_countFew)
y_train=y_train.drop(index_countFew)
print len(X_train_nanRate),len(y_train)

### 3) Handling missing data (for dense train data)

In [ ]:
X_train_nanRate.isnull()
print 'the number of nan value in each column:'
X_train_nanRate.isnull().sum()

In [ ]:
# have a look at the observation whose featuer '3' is null
X_train_nanRate[X_train_nanRate['3'].isnull()]

### the ways to handling the miss values:
#### 1) we can delete the observations with nan value directly
#### 2)we can inputer the the nan value with something else basen on the knowledge of the dataset
#### 3) but more rational way is to treat the data as sparse data, we can improve the model by treating it as a sparse matrix

In [ ]:
print X_train_nanRate.shape
print X_train_nanRate.dropna(how='any').shape
print X_train_nanRate.dropna(how='all').shape

In [ ]:
### we fill the nan with 0000 as a new value for each feature ,then the estimator can handle them
X_train_nanRate.fillna(value='0000',inplace=True)

### (4)Encoding the categorical vriables

In [ ]:
#use the get_dummies funtion to tranform the categorical variables into dummy variable with binary values
#Create a list of features to dummy
todummy_list = list(X_train_nanRate.columns)
# Function to dummy all the categorical variables used for modeling
def dummy_df(df, todummy_list):
    for x in todummy_list:
        dummies = pd.get_dummies(df[x], prefix=x, dummy_na=False)
        df = df.drop(x, 1)
        df = pd.concat([df, dummies], axis=1)
    return df

#better than just pd.get_dummies

# X_train_dummies= pd.get_dummies(X_train)
X_train_nanRate_dummies = dummy_df(X_train_nanRate,todummy_list)
print X_train_nanRate.shape
print X_train_nanRate_dummies.shape
#there is nan value in this dense train data, we need to treat this problem when construct the estimator with imputer method

In [ ]:
X_train_nanRate_dummies.head()

In [ ]:
#anothter way : use LabelEncoder to transform the categorical varialble to 0 to n-class-1 (or 1 to n-class)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X_train_nanRate_label =pd.DataFrame()

for x in X_train_nanRate.columns:
    # here we alread treat the nan value in the X_train as one kind of value
    X_train_nanRate_label[x] = le.fit_transform(X_train_nanRate[x].astype('str'))

X_train_nanRate_label.shape
#X_train_label    

In [ ]:
#then use OneHotEncode to encode the label into dummy variable 
from sklearn.preprocessing import OneHotEncoder 
enc = OneHotEncoder()
enc.fit(X_train_nanRate_label)  
X_train_nanRate_encoded=enc.transform(X_train_nanRate_label).toarray()

In [ ]:
X_train_nanRate_encoded[0,:]
X_train_nanRate_encoded.shape

In [ ]:
#the dummy train data is same with the encoded train data
X_train_nanRate_encoded[0:1]

In [ ]:
# NO need !!!
# #use LabelEncoder to transform the response varialble to 0 to n-class-1 (or 1 to n-class)
# y_le = LabelEncoder()
# y_le.fit(y_train.astype('str'))
# y_le.classes_
# y_train_label = y_le.transform(y_train)


# Pre-Classification by the vin code information and platform imformation
### 1. vin- code NO1-3: to choose which vehicle company it belong：
lsj : 中国上海荣威
### 2. plateform : to choose which plateform it  belong
荣威550：
### more information : to pre-classify the vin code into different prediction model?


raw_data.head()

# Part2: Model  selection 
### we want to try other models to find the best one

### Agenda:
1.  parameter tuning for each model (Cross-validation or train/test split)

  **Goal:** Select the best tuning parameters (aka "hyperparameters") for each on the iris dataset
  - **sklearn.cross_validation.cross_val_score** 
      
      loops for single parameter
      
  - **sklearn.grid_search.GridSearchCV ** More efficient parameter tuning 
    
     Allows you to define a **grid of parameters** that will be **searched** using K-fold cross-validation
  
  - **sklearn.grid_search.RandomizedSearchCV** Reducing computational expense
  
     Searching many different parameters at once may be computationally infeasible
    
     `RandomizedSearchCV` searches a subset of the parameters, and you control the computational "budget"
     
2. model selection (Cross-validation or train/test split)

  **Goal:** Compare the best models with each other
  
3. feature selection (Cross-validation)

   **Tips**:Normally, feature engineering and selection occurs **before** cross-validation 
         if the data have lots of features,in computing practice we need to selection the features when preprocessing 

   **Goal**: Select whether somefeatures should be included in the model



## 2.1 The 1st mode :Random Forest 
### 1) Tuning the parameters with cross-validation
 -- A search consists of:
 - an estimator (regressor or classifier such as sklearn.svm.SVC());
 - a parameter space;
 - a method for searching or sampling candidates;
 - a cross-validation scheme; and
 - a score function.

#### Construct a estimator

In [57]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer


# ##becaue when label the featrues into intergers, the nan value in each feature has benn already transfromed into one vule,so we don't need
# ##the imputer function actually.
# estimator = Pipeline([("imputer", Imputer(missing_values=np.nan,
#                                           strategy="most_frequent",
#                                           axis=0)),
#                       ("forest", RandomForestClassifier(random_state=0,
#                                                        n_estimators=num_tree,))])

#### Model selction with train-test split

In [ ]:
num_tree =100
#instanciate the model
estimator = RandomForestClassifier(n_estimators=num_tree,n_jobs=2)
#split the train data into two parts
train_X,test_X,train_y,test_y = train_test_split( X_train_nanRate_dummies,y_train,)

#fit the estimator on the develpment part
estimator.fit(train_X,train_y)

#predict on the evalation part
predicted =estimator.predict(test_X)

# evaluate the estimator 
accuracy_score(predicted,test_y)

#### trainning the model on different size of trees

In [ ]:
randomForestClf_accuracy_scores=[]
treeSize_range = [100,500,100]
for num_tree in treeSize_range:
    estimator = RandomForestClassifier(n_estimators=num_tree,n_jobs=2)
    #split the train data into two parts
    train_X,test_X,train_y,test_y = train_test_split( X_train_nanRate_dummies,y_train,)

    #fit the estimator on the develpment part
    estimator.fit(train_X,train_y)

    #predict on the evalation part
    predicted =estimator.predict(test_X)
    # evaluate the estimator 
    acc = accuracy_score(predicted,test_y)
    
    randomForestClf_accuracy_scores.append(acc)

### plot the scores

In [ ]:
# plot the relationship between K and testing accuracy
plt.plot(treeSize_range, randomForestClf_accuracy_scores)
plt.xlabel('treeSize for randomForestClf')
plt.ylabel('Testing Accuracy')

#### Question : is there a problem just substitute the nan value with 'mean'??

#### Acccessing the estimator with cross validation
but this is too much time consumping for single computer!!!

In [ ]:
# #using the X_train_dummies as train data of features
# from sklearn.cross_validation  import cross_val_score
# import sklearn.metrics
# # # score = cross_val_score(estimator, X_train_dummies, y_train,scoring=sklearn.metrics.accuracy_score).mean()
# score = cross_val_score(estimator, X_train_nanRate_dummies, y_train,n_jobs=2).mean()
# score
# # score = cross_val_score(estimator, X_train_nanRate_encoded, y_train,n_jobs=2).mean()

C:\ProgramData\Anaconda3\envs\python27\lib\site-packages\sklearn\cross_validation.py:516: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=3.
  % (min_labels, self.n_folds)), Warning)


0.62501312904056061

In [90]:
vin='LSJW26H38AS072686'
raw_data[raw_data['RACK_NO']==vin]['FEATURE_LIST'].values[0]

'20093A176ASA76H3A00UA10SA40DB00LC00HC06CC08TC09CC10HC40VC91XD00GD01ME00TE55AF10LF11BF15KG10NH10DH12BH15AJ00FJ05EJ10PJ15AK19AK202K61BL10AL15AL18CL20AM05DM10DM19PM26AM27CM35AM40LM42CM43CN00AN06BN101N13MN22AN50LN62MN68AQ00BQ123Q19RS00PS15DS16DS35SS40PS42AT11AU00LU01UU039U04AU058U11XU35RZ10C'

### Tunning the parameters of RandomForestClassfier
##### two variables： n_estimators && max_features
the number of tree and the random partion of variables && the size of the random subsets of features to consider when splitting a node
max_features=sqrt(n_features) default for classification tasks (where n_features is the number of features in the data)


#### Parallizition :n_jobs=k 
     then computations are partitioned into k jobs, and run on k cores of the machine. 
     If n_jobs=-1 then all cores available on the machine are used.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation  import cross_val_score
import sklearn.metrics
num_trees = range(100,1001,100)
n_features = train_data.shape[1]
RandomForestClf_scores =[]

for n in num_trees:
    num_tree = n
    estimator = RandomForestClassifier(n_estimators=num_tree,n_job=1)
    score = cross_val_score(estimator, X_train_dummies,scoring=sklearn.metrics.accuracy_score, y_train,n_jobs=1).mean()
    RandomForestClf_scores.append(score)

In [ ]:
#plot the performance for each num of tree
RandomForestClf_scores

#### Feature Importances
In practice those estimates are stored as an attribute named feature_importances_ on the fitted model. 
This is an array with shape (n_features,) whose values are positive and sum to 1.0. 
The higher the value, the more important is the contribution of the matching feature to the prediction function.

### 2) Model Acccessing 

In [1]:
## 2.2 AdaBoost
### 1) Tuning the parameter with cross-validation

# from sklearn.ensemble import AdaBoostClassifier
# from sklearn.cross_validation  import cross_val_score

# num_trees = range(100,2001,100)
# n_features = train_data.shape[1]
# AdaBoostClf_scores =[]

# for n in num_trees:
    
#     num_tree = n
#     estimator = AdaBoostClassifier()
#     score = cross_val_score(estimator, X_train_dummies, y_train,n_jobs=-1).mean()
#     AdaBoostClf_scores.append(score)

C:\ProgramData\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## 2.2 Gradient Tree Boosting¶

In [3]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV

 Classification with more than 2 classes requires the induction of n_classes regression trees at each iteration, thus, the total number of induced trees equals n_classes * n_estimators. For datasets with a large number of classes we strongly recommend to use RandomForestClassifier as an alternative to GradientBoostingClassifier .

In [7]:
# ### Fit classifier with interation 500 times
from sklearn.model_selection import train_test_split
train_X, test_X, train_y,test_y = train_test_split(X_train_nanRate_dummies,y_train,test_size=0.2)
GB_clf = GradientBoostingClassifier(n_estimators=500,learning_rate=0.1)
GB_clf.fit(train_X,train_y)
GB_clf.score(test_X,test_y)


In [ ]:
####### Tunning the parameters
# #set the parameters we want to tunning
# params = {'n_estimators': [100,500,1000], 'max_depth': [1:4], 'subsample': 0.5,
#           'learning_rate': 0.01, 'min_samples_leaf': 1}

# ####instance the estimator
# GBclf = GradientBoostingClassifier(**params)

# ###using the randomizedSearchCV
# RandomizedSearchCV(GB_clf,param_distributions=params,n_jobs=2)

# ### check the performance

## 2.3 Model: XG booosting

# Part3: Model  evaluation
Use a model evaluation procedure to estimate how well a model will generalize to out-of-sample data

Requires a model evaluation metric to quantify the model performance
## Agenda:
after we have selection the model,the parameter and features in part 1 and part2

1. Model evaluation (Train/test split,K-fold cross-validation)

2. Model evaluation metrics
 - Regression problems: Mean Absolute Error, Mean Squared Error, Root Mean Squared Error
 - Classification problems: 
 
       A) Classification accuracy
   
       B) confusion matrix
   
       C) ROC curve adn Area Under the Curve (AUC) differ from classification accuracy
       
       **Question:** Wouldn't it be nice if we could see how sensitivity and specificity are affected by various thresholds, without actually changing the threshold?
       **Answer:** Plot the ROC curve!


# Testing

In [91]:
#bulid a funciton to interpret a vin code into a feature list by our EPC system
def vinInterpretedToFeatureList(vin):
    #samle is a feature list
    temp_data = pd.DataFrame(np.nan, index=[0],columns=dict_column)
    temp_data_CN = pd.DataFrame(np.nan, index=[0],columns=dict_column)
    
    feature_list = raw_data[raw_data['RACK_NO']==vin]['FEATURE_LIST'].values[0]
    #feature_list = raw_data['FEATURE_LIST'][279]
    # print feature_list
    # print temp_data
    # print type(dict_column)
    # print type(feature_list[0])
    count =0
    for xx in range(0,len(feature_list),4):

        if (feature_list[xx] in ['2','3','5','6','7']) and (count <=6):
            count = count+1
            feature =feature_list[xx]
            feature_code =feature_list[xx:xx+4]
            #print xx,count,feature_code
            #print feature_code

            if tm_feature[tm_feature['FEATURE_CODE']==feature_code].values.shape[0]==0:
                #some feature_code like 76E3 in 279th observation ,but not exsit in the tm_feature['FEATURE_CODE], we assume it is just the correce 
                #feature_code,but fail to find the chinese feature name
                if tm_feature[tm_feature['FEATURE_CODE']==feature_code[0:3]].values.shape[0]==0:
                    temp_data[feature] = np.nan
                    temp_data_CN[feature]= np.nan

                #some feature_code lke G10N in 0th observation, but not exsit in the tm_feature['FEATURE_CODE],
                #we assume it is the same as G10    
                else:
                    temp_data[feature] = feature_code[0:3]
                    temp_data_CN[feature]= feature_code[0:3]

            #print feature_code
            else:
                temp_data[feature] = feature_code
                #print feature_code, tm_feature[tm_feature['FEATURE_CODE']==feature_code]
                temp_data_CN[feature]= tm_feature[tm_feature['FEATURE_CODE']==feature_code].values[(0,4)]
                #.values which change a data frame into np array

        elif any(feature_list[xx:xx+3] == x for x in dict_column):
            #print any(feature_list[xx:xx+3] == x for x in dict_column)
            feature = feature_list[xx:xx+3]
            feature_code = feature_list[xx:xx+4]

            if tm_feature[tm_feature['FEATURE_CODE']==feature_code].values.shape[0]==0:
                #some feature_code like 76E3 in 279th observation ,but not exsit in the tm_feature['FEATURE_CODE], we assume it is just the correce 
                #feature_code,but fail to find the chinese feature name
                if tm_feature[tm_feature['FEATURE_CODE']==feature_code[0:3]].values.shape[0]==0:
                    temp_data[feature] = np.nan
                    temp_data_CN[feature]= np.nan


                #some feature_code lke G10N in 0th observation, but not exsit in the tm_feature['FEATURE_CODE],
                #we assume it is the same as G10    
                else:
                    temp_data[feature] = feature_code[0:3]
                    temp_data_CN[feature]= feature_code[0:3]

            #print feature_code
            else:
                temp_data[feature] = feature_code
                #print feature_code, tm_feature[tm_feature['FEATURE_CODE']==feature_code]
                temp_data_CN[feature]= tm_feature[tm_feature['FEATURE_CODE']==feature_code].values[(0,4)]
                #.values which change a data frame into np array

    return temp_data

vin='LSJW26H38AS072686'
vinInterpretedToFeatureList(vin)
# we can use other 'FEATURE_CN_CHINA' OR 'NUM' HERE   


,2,3,5,6,7,A00,A01,A02,A03,A05,...,U35,U40,U41,U42,Z00,Z10,Z15,Z16,Z90,Z99
0,2009,3A17,NaN,6ASA,76H3,A00U,NaN,NaN,NaN,NaN,...,U35R,NaN,NaN,NaN,NaN,Z10C,NaN,NaN,NaN,NaN


In [92]:
def vinInterpretedToFeatureLables(vin):
    temp_data =vinInterpretedToFeatureList(vin)
    temp_label=pd.DataFrame()
    for x in X_train.columns:
        le = LabelEncoder()
        #we need to use the train data here to transform the data, but acturally we just need the dictory of the feature to transform
        le.fit(X_train[x].astype('str'))
        temp_label[x] = le.transform(temp_data[x])
    return temp_label
        

vinInterpretedToFeatureLables(vin)    
    
    

,2,3,6,7,A00,A10,A40,B00,C00,C06,...,Q19,S00,S15,S35,S40,T11,U01,U05,U11,U35
0,2,0,0,27,11,3,0,0,7,0,...,0,5,3,0,1,0,1,3,1,0


In [93]:
# trainning the estimator with the full train data
clf = RandomForestClassifier(random_state=11,n_estimators=100)
estimator = clf.fit(X_train_label, y_train)


In [94]:
#predict with the estimator
test_vin_feature =vinInterpretedToFeatureLables(vin)
estimator.predict(test_vin_feature)

array(['RWAABD0022'], dtype=object)

In [95]:
def vinEstimator(vin):
    
    test_vin_feature =vinInterpretedToFeatureLables(vin)
    vehicle_code = estimator.predict(test_vin_feature)
    return vehicle_code
    

# input

In [96]:
# while True: 
#     vin_code= raw_input('please input the 17 bits vin code: ') # why I can't use the function input('')
#     #vin_code ='LSJW26H38AS072686'
#     if vin_code == 'quit' or vin_code == 'q':
#         break
# #         print 'hi'

#     elif len(vin_code) != 17:
#         print 'the vin code is not correct'
#     else:
#         vehicle_code_predicted = vinEstimator(vin_code)
#         vehicle_code = raw_data[raw_data['RACK_NO']==vin_code]['VEHICLE_CODE'].values[0] #get the value from a series
#         print 'the predicted vehicle code is '+ vehicle_code_predicted
#         print 'the actual vehicle code is '+vehicle_code
#         if vehicle_code_predicted == vehicle_code:
#             print 'Wow,congratulation!! your prediction is correct!'
#         else:
#             print 'It is a pity, your prediction is wrong!!!'+'\n'
#             print 'if you want to quit,please input: quit or q'+'\n'


In [97]:
vinEstimator('LSJW26H33AS079268')
vinEstimator('LSJW26H36AS082617')

array(['RWAABD0022'], dtype=object)

In [135]:
# raw_data[0:2000]

# improving the predict model
### one way:     adding into more features from the 17 bits vin code
### another way: using the information from the 17 bits vin code to pre-classify the model
### one way:     selection better classifitor: such as SVM, Nural network,(I dont thing knn will improve the performance,logistics                   classification is for linear boundary) 
                                                                    

# Part3: Model persistence
After training a scikit-learn model, it is desirable to have a way to persist the model for future use without having to retrain. 
The following section gives you an example of how to persist a model with pickle. We’ll also review a few security and maintainability issues when working with pickle serialization.